In [3]:
import zipfile
import os

# Fire/Smoke images extract
with zipfile.ZipFile("fire_images.zip", 'r') as zip_ref:
    zip_ref.extractall("fire_images")  # ye folder create hoke saari images yahan aa jayengi

# Non-fire images extract
with zipfile.ZipFile("non_fire_images.zip", 'r') as zip_ref:
    zip_ref.extractall("non_fire_images")

# Check first few files
print("Fire images:", os.listdir("fire_images")[:5])
print("Non-fire images:", os.listdir("non_fire_images")[:5])


Fire images: ['fire_images']
Non-fire images: ['non_fire_images']


In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt


In [5]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
print(tf.__version__)


2.20.0


In [2]:
import zipfile
import os

# Paths to your zip files
fire_zip = 'fire_images.zip'
non_fire_zip = 'non_fire_images.zip'

# Create dataset folder if it doesn't exist
if not os.path.exists('dataset'):
    os.makedirs('dataset')

# Unzip fire_images.zip into dataset/fire_images
with zipfile.ZipFile(fire_zip, 'r') as zip_ref:
    zip_ref.extractall('dataset/fire_images')

# Unzip non_fire_images.zip into dataset/non_fire_images
with zipfile.ZipFile(non_fire_zip, 'r') as zip_ref:
    zip_ref.extractall('dataset/non_fire_images')

print("Unzipping done. Folder structure created inside 'dataset' folder.")


Unzipping done. Folder structure created inside 'dataset' folder.


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 128, 128
batch_size = 32

# Prepare ImageDataGenerator to rescale images and split dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% data for validation
)

train_generator = train_datagen.flow_from_directory(
    directory='dataset',   # folder where your fire_images and non_fire_images are
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True,
    seed=123
)

validation_generator = train_datagen.flow_from_directory(
    directory='dataset',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=True,
    seed=123
)


Found 1424 images belonging to 2 classes.
Found 355 images belonging to 2 classes.


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Set image dimensions
img_height, img_width = 128, 128

# Define the CNN model
model = Sequential([
    Input(shape=(img_height, img_width, 3)),  # Input layer with correct image shape
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary output layer for fire/smoke classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model for 10 epochs with your train and validation data generators
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)



Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6889 - loss: 0.5838

C:\ProgramData\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


45/45 ━━━━━━━━━━━━━━━━━━━━ 263s 6s/step - accuracy: 0.7978 - loss: 0.4263 - val_accuracy: 0.8056 - val_loss: 0.4442
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 194s 4s/step - accuracy: 0.8848 - loss: 0.2876 - val_accuracy: 0.8451 - val_loss: 0.3917
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 198s 4s/step - accuracy: 0.9185 - loss: 0.2257 - val_accuracy: 0.8282 - val_loss: 0.4101
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 171s 4s/step - accuracy: 0.9256 - loss: 0.2146 - val_accuracy: 0.8535 - val_loss: 0.3853
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 215s 5s/step - accuracy: 0.9235 - loss: 0.1988 - val_accuracy: 0.8451 - val_loss: 0.4374
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 206s 5s/step - accuracy: 0.9333 - loss: 0.1636 - val_accuracy: 0.8592 - val_loss: 0.3827
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 172s 4s/step - accuracy: 0.9473 - loss: 0.1405 - val_accuracy: 0.8732 - val_loss: 0.5424
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 389s 9s/step - accuracy: 0.9431 - loss: 0.1228 - val_accuracy: 0.8394 - val_loss: 0.

In [8]:
model.save('model.keras')



In [9]:
# Evaluate model performance on validation dataset
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss:.4f}')
print(f'Validation Accuracy: {accuracy:.4f}')


12/12 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.8704 - loss: 0.5541
Validation Loss: 0.5541
Validation Accuracy: 0.8704


In [ ]:

import winsound
winsound.PlaySound(r'C:\Users\Aayush\Downloads\siren.wav', winsound.SND_FILENAME)



In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from playsound import playsound
import threading

# Load your trained model
model = load_model('model.keras', compile=False)

# Webcam video capture
cap = cv2.VideoCapture(0)  # 0 is default webcam

# Image size expected by your model
img_height, img_width = 128, 128

alerting = False

import winsound

def play_alert_sound():
    global alerting
    winsound.PlaySound(r'C:\Users\Aayush\Downloads\siren2.wav', winsound.SND_FILENAME)
    alerting = False


while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame to model input size
    img = cv2.resize(frame, (img_width, img_height))
    img = img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    # Predict fire or smoke presence
    prediction = model.predict(img)[0][0]

    # Threshold for fire detection
    if prediction > 0.8:
        label = f"Fire/Smoke detected ({prediction*100:.1f}%)"
        color = (0, 0, 255)  # red box for alert

        # Play alert sound in separate thread (to avoid blocking)
        if not alerting:
            alerting = True
            threading.Thread(target=play_alert_sound).start()

    else:
        label = "No Fire/Smoke detected"
        color = (0, 255, 0)  # green box

    # Show label and rectangle on frame
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.8, color, 2)
    cv2.rectangle(frame, (5, 5), (frame.shape[1]-5, frame.shape[0]-5), color, 3)

    # Display the video frame
    cv2.imshow('Fire and Smoke Detection', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━